In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.ensemble import RandomForestRegressor
from sklearn import cross_validation
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor

In [2]:
sales = pd.read_csv("../data/train.csv")
sales = sales.sample(frac=0.1)
stores = pd.read_csv("../data/store.csv")
sales = sales.merge(stores, on='Store', how='left')

sales['Date'] = pd.to_datetime(sales.Date)
sales['Year'] = sales.Date.dt.year
sales['Month'] = sales.Date.dt.month
sales['Day'] = sales.Date.dt.day
sales['DayOfWeek'] = sales.Date.dt.dayofweek
sales['WeekOfYear'] = sales.Date.dt.weekofyear

monthMapper = {'Jan': 1, 'Feb':2, 'Mar':3, 'Apr':4, 'May':5, 'Jun':6, \
             'Jul':7, 'Aug':8, 'Sept':9, 'Oct':10, 'Nov':11, 'Dec':12}

print "reducing to: " + str(sales.shape[0]) + " rows"

reducing to: 1017209 rows


/Users/jason.xie/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
sales = pd.concat([sales, pd.get_dummies(sales['StateHoliday'], prefix='StateHoliday').iloc[:,1:]], axis=1)
sales = pd.concat([sales, pd.get_dummies(sales['Assortment'], prefix='Assortment')], axis=1)
sales = pd.concat([sales, pd.get_dummies(sales['StoreType'], prefix='StoreType')], axis=1)
sales = pd.concat([sales, pd.get_dummies(sales['Year'], prefix='Year')], axis=1)
sales = pd.concat([sales, pd.get_dummies(sales['Month'], prefix='Month')], axis=1)
sales = pd.concat([sales, pd.get_dummies(sales['Day'], prefix='Day')], axis=1)
sales = pd.concat([sales, pd.get_dummies(sales['DayOfWeek'], prefix='DayOfWeek')], axis=1)
sales = pd.concat([sales, pd.get_dummies(sales['WeekOfYear'], prefix='WeekOfYear')], axis=1)

In [4]:
sales = sales.drop(['PromoInterval', 'Date', 'Customers', 'StateHoliday', 'Assortment', 'Year', 'Month', 'Day', 'DayOfWeek', 'WeekOfYear', 'StoreType'], axis=1)

In [5]:
X, y = sales.drop(['Sales'], axis=1), sales['Sales']
X['CompetitionDistance'] = X['CompetitionDistance'].fillna(-1)
X['CompetitionOpenSinceMonth'] = X['CompetitionOpenSinceMonth'].fillna(-1)
X['CompetitionOpenSinceYear'] = X['CompetitionOpenSinceYear'].fillna(-1)
X['Promo2SinceWeek'] = X['Promo2SinceWeek'].fillna(-1)
X['Promo2SinceYear'] = X['Promo2SinceYear'].fillna(-1)

In [6]:
evals = pd.read_csv("../data/test.csv")
evals = evals.merge(stores, on='Store', how='left')
print "eval has: " + str(evals.shape[0]) + " rows"
evals['Date'] = pd.to_datetime(evals.Date)
evals['Year'] = evals.Date.dt.year
evals['Month'] = evals.Date.dt.month
evals['Day'] = evals.Date.dt.day
evals['DayOfWeek'] = evals.Date.dt.dayofweek
evals['WeekOfYear'] = evals.Date.dt.weekofyear
evals = pd.concat([evals, pd.get_dummies(evals['StateHoliday'], prefix='StateHoliday')], axis=1)
evals = pd.concat([evals, pd.get_dummies(evals['Assortment'], prefix='Assortment')], axis=1)
evals = pd.concat([evals, pd.get_dummies(evals['StoreType'], prefix='StoreType')], axis=1)
evals = pd.concat([evals, pd.get_dummies(evals['Year'], prefix='Year')], axis=1)
evals = pd.concat([evals, pd.get_dummies(evals['Month'], prefix='Month')], axis=1)
evals = pd.concat([evals, pd.get_dummies(evals['Day'], prefix='Day')], axis=1)
evals = pd.concat([evals, pd.get_dummies(evals['DayOfWeek'], prefix='DayOfWeek')], axis=1)
evals = pd.concat([evals, pd.get_dummies(evals['WeekOfYear'], prefix='WeekOfYear')], axis=1)
evals = evals.drop(['PromoInterval', 'Date', 'StateHoliday', 'Assortment', 'Year', 'Month', 'Day', 'DayOfWeek', 'WeekOfYear', 'StoreType'], axis=1)

for column in X.columns:
    if column not in evals.columns:
        evals[column] = 0
X_eval = evals
for column in X_eval.columns:
    X_eval[column] = X_eval[column].fillna(-1)

eval has: 41088 rows


In [7]:
names = X.columns

In [8]:
X.sample(3)

,Store,Open,Promo,SchoolHoliday,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,...,WeekOfYear_43,WeekOfYear_44,WeekOfYear_45,WeekOfYear_46,WeekOfYear_47,WeekOfYear_48,WeekOfYear_49,WeekOfYear_50,WeekOfYear_51,WeekOfYear_52
463799,745,1,0,0,17650,11,2013,1,37,2009,...,0,0,0,0,0,0,0,0,0,0
587625,806,1,1,0,260,-1,-1,1,44,2010,...,0,0,0,0,0,0,0,0,0,0
290036,433,1,1,0,30030,11,2010,0,-1,-1,...,0,0,1,0,0,0,0,0,0,0


In [9]:
y.sample(3)

597834       0
966219    5314
295048    9128
Name: Sales, dtype: int64

In [10]:
X, y = X.values, y.values
X, X_test, y, y_test = cross_validation.train_test_split(X, y, test_size=0.1)

In [11]:
# note the variable 'Open' probably have 0 importance because the # of Customers already encapsulates the 'Open' information

In [12]:
adaboost = AdaBoostRegressor(DecisionTreeRegressor(max_depth=15), n_estimators=50, loss='linear', learning_rate=1)

In [ ]:
adaboost.fit(X, y)

In [ ]:
adaboost.score(X_test, y_test)

In [ ]:
print sorted(zip(map(lambda x: round(x, 4), adaboost.feature_importances_), names), 
             reverse=True)

In [ ]:
# X_eval = X_eval.sample(1)
X_input = X_eval.drop(['Id'], axis=1).values
f = open('../data/submission.csv', 'wb')
f.write('"Id","Sales"\n')
for i in range(0, X_input.shape[0]):
    f.write(str(i + 1))
    f.write(",")
    f.write(str(int(adaboost.predict(X_input[i])[0])))
    if i != X_input.shape[0] - 1:
        f.write("\n")
f.close()